### Tracking of Add-on from PNOC File

In [136]:
# Libraries
import pandas as pd
import glob
import json
import warnings

In [ ]:
# Import of current addons spreadsheet from pnoc file.


# Import JSON file containing the mapping of addons to towers.
with open(r'supports/addons_tower.json', 'r',encoding='utf-8') as json_file:
    addons = json.load(json_file)

# Transform the json structure into a dictionary for easy mapping.
addons_dict = {}
for key, values in addons.items():
    for item in values:
        addons_dict[item['Commodity']] = key

#input_path = input("Enter the path to the current pnoc file: ").strip('"')  # leave it for final version.
path = glob.glob(r"")  # leave it for testing. add the path later.

def addon_type(x,path):
  all_files = []        # In this example the list was included inside the function. Otherwise it would mix data from the two call types.
  if x == "Add-ons":
    for files in path:
      pnoc_addons = pd.read_excel(files,engine="openpyxl",
                                   sheet_name="Add-ons",
                                     skiprows=1,
                                       usecols="B:E,G:O")
      all_files.append(pnoc_addons)
      
      
  elif x == "Adj":
    for files in path:
      pnoc_adj = pd.read_excel(files,engine="openpyxl",
                                sheet_name="Adjustments",
                                skiprows=2, 
                                usecols="B:E,AS:BA")
      all_files.append(pnoc_adj)
  
  if all_files:
        return pd.concat(all_files, ignore_index=True)    

# Read of the json file containing the list of towers and their respective communities.


In [138]:
# Ignore specific warnings from openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
df_addons = addon_type("Add-ons",path)
# Remove unnecessary columns and rows.
df_addons = df_addons.dropna(subset=['Update']) # type:ignore
# Rename columns for clarity.
df_addons = df_addons.rename(columns={
    'Biscuits': 'Biscuits US',
    'Confy': 'Confy US',
    'Biscuits.1': 'Biscuits CA',
    'Confy.1': 'Confy CA',
    'Main comment': 'Comment'})

# Column 'Update' to datetime format.
df_addons['Update'] = pd.to_datetime(df_addons['Update'], format='%B %Y') + pd.offsets.MonthBegin(0)
df_addons['Update'] = df_addons['Update'].dt.date
# Map tower information from the json file to the dataframe.
df_addons.insert(0,'Tower',df_addons['Commodity'].map(addons_dict))

# Fill NaN values in 'Driver' and 'Comment' columns with 'No Comment'.
df_addons[['Driver', 'Comment']] = df_addons[['Driver', 'Comment']].fillna('No Comment')

# Pivot tables for summary views.
summary_addons = df_addons.pivot_table(
    index=['Tower','Commodity', 'Driver', 'Comment'],
    columns='Update',
    values='NA',
    aggfunc='sum'
).reset_index()
addons_pivot = df_addons.pivot_table(
    index=['Tower','Commodity'],
    columns='Update',
    values='NA',
    aggfunc='sum'
)


In [139]:
df_adj = addon_type("Adj",path)
# Remove unnecessary columns and rows.
df_adj = df_adj.dropna(subset=['Update']) # type:ignore
# Rename columns for clarity.
df_adj = df_adj.rename(columns={
    'Biscuits.8': 'Biscuits US',
    'Confy.8': 'Confy US',
    'US.4': 'US',
    'Biscuits.9': 'Biscuits CA',
    'Confy.9': 'Confy CA',
    'CAN.4': 'CA',
    'cliff.1' : 'Cliff',
    'Main comment.4': 'Comment',
    'NA.4': 'NA'}
    )

# Column 'Update' to datetime format.
df_adj['Update'] = pd.to_datetime(df_adj['Update'], format='%B %Y') + pd.offsets.MonthBegin(0)
df_adj['Update'] = df_adj['Update'].dt.date
# Map tower information from the json file to the dataframe.
df_adj.insert(0,'Tower',df_adj['Commodity'].map(addons_dict))

# Fill NaN values in 'Driver' and 'Comment' columns with 'No Comment'.
df_adj[['Driver', 'Comment']] = df_adj[['Driver', 'Comment']].fillna('No Comment')

# Pivot tables for summary views.
summary_adj = df_adj.pivot_table(
    index=['Tower','Commodity', 'Driver', 'Comment'],
    columns='Update',
    values='NA',
    aggfunc='sum'
).reset_index()
adj_pivot = df_adj.pivot_table(
    index=['Tower','Commodity'],
    columns='Update',
    values='NA',
    aggfunc='sum'
)

In [ ]:
path_archive = r""  # leave it for testing. Add the path later.

with pd.ExcelWriter(path_archive) as writer:
    addons_pivot.to_excel(writer, sheet_name='Addons Summary by Tower')
    summary_addons.to_excel(writer, sheet_name='Addons Summary by Commodity', index=False)
    adj_pivot.to_excel(writer, sheet_name='Adj Summary by Tower')
    summary_adj.to_excel(writer, sheet_name='Adj Summary by Commodity', index=False)